In [1]:
%matplotlib inline

import random
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import pystan 
import pandas as pd
import seaborn as sns
import mathz
import statistics as st
from matplotlib.backends.backend_pdf import PdfPages
import pickle
from tqdm.notebook import tqdm as tqdm

In [2]:
%matplotlib inline

import pystan 
import pickle

#Stan fitting
model = """
data {
	int N;    
	int K;    //
	vector[N] Nt;     
	real t[N];
	int<lower=1, upper=K> strain[N];    
	real<lower=3.0, upper=3.8> pH[N];  
	real Nu;
}	

parameters {
	vector<lower=4, upper=7.5>[K] I;
	real<lower=4, upper=7.5> I0;  
	real<lower=0> s_I;
	real<lower=0, upper=1> s_Nt;
	vector [4] abef[K];
	vector [4] abef0;
	cholesky_factor_corr[4] corr_chol;
	vector<lower=0>[4] sigma_vec;
}
transformed parameters{
	vector[K] a;
	vector[K] b;
	vector[K] e;
	vector[K] f;
	cholesky_factor_cov[4] cov_chol;
	vector [N] M;
	vector [N] d;
	vector [N] p;
	for(k in 1:K){
		a[k]=abef[k,1];
		b[k]=abef[k,2];
		e[k]=abef[k,3];
		f[k]=abef[k,4];
	}
	cov_chol = diag_pre_multiply(sigma_vec,corr_chol);
	for(n in 1:N)
	{
		d[n] = exp(a[strain[n]]*pH[n]+b[strain[n]]);
		p[n] = exp(e[strain[n]]*pH[n]+f[strain[n]]);
		M[n] = I[strain[n]] -(t[n]/d[n])^p[n];
	}
}
model {
	I ~ normal(I0, s_I);
	corr_chol ~ lkj_corr_cholesky(Nu);
	abef ~ multi_normal_cholesky(abef0,cov_chol);
	Nt ~ normal(M, s_Nt);
}
generated quantities {
	matrix[4,4] corr;
	matrix[4,4] cov;
	vector[N] log_lik;
	corr = multiply_lower_tri_self_transpose(corr_chol);
	cov = multiply_lower_tri_self_transpose(cov_chol);
	for(n in 1:N){
		log_lik[n] = normal_lpdf(Nt[n]|I[strain[n]] -(t[n]/d[n])^p[n],s_Nt);
	}
}
"""
sm = pystan.StanModel(model_code=model)

with open('between-strain model.pkl', 'wb') as f:

    pickle.dump(sm, f)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_2c516704ae2235d80690cd7e227664ca NOW.


In [3]:
# Reduction data Import
datum = pd.read_csv('Data/Gastric Data/allcamdata.csv')
print(datum)

       TIME         N   pH  STRAIN  pHindex
0       0.0  5.949390  3.0       1        1
1       0.0  5.707570  3.0       1        1
2       0.0  5.819544  3.0       1        1
3       1.5  6.060698  3.0       1        1
4       1.5  6.037426  3.0       1        1
...     ...       ...  ...     ...      ...
1143  210.0  4.287802  3.8      11        5
1144  210.0  4.385606  3.8      11        5
1145  240.0  4.334454  3.8      11        5
1146  240.0  3.991226  3.8      11        5
1147  240.0  3.518514  3.8      11        5

[1148 rows x 5 columns]


In [4]:
# Sampling Baysian model of gastric reduction
sm_GR_between_strain = pickle.load(open('between-strain model.pkl', 'rb'))
fit_nuts_between_strain = sm_GR_between_strain.sampling(
    data=dict(t = datum["TIME"], Nt = datum["N"], strain = datum["STRAIN"], pH = datum["pH"], N = len(datum["TIME"]), K=11, Nu=1), 
    iter=5000, chains=4, thin=1, warmup=2500, seed=123, control = dict(adapt_delta=0.8, max_treedepth=15)
    )

print(fit_nuts_between_strain)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


Inference for Stan model: anon_model_2c516704ae2235d80690cd7e227664ca.
4 chains, each with iter=5000; warmup=2500; thin=1; 
post-warmup draws per chain=2500, total post-warmup draws=10000.

                 mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
I[1]             5.97  4.3e-3   0.14   5.67   5.87   5.98   6.08    6.2   1098    1.0
I[2]             6.14  9.9e-4   0.09   5.96   6.09   6.15    6.2   6.31   7557    1.0
I[3]             6.19  1.1e-3   0.09   6.02   6.13   6.19   6.25    6.4   6870    1.0
I[4]             6.18  1.0e-3   0.09   6.01   6.13   6.18   6.24   6.38   8386    1.0
I[5]             6.16  8.5e-4   0.09   5.98    6.1   6.16   6.22   6.34  11014    1.0
I[6]             6.16  8.2e-4   0.09   5.99    6.1   6.16   6.21   6.34  11503    1.0
I[7]             6.25  2.1e-3    0.1   6.07   6.17   6.24   6.31   6.47   2340    1.0
I[8]             6.13  9.7e-4   0.09   5.95   6.08   6.14   6.19    6.3   8616    1.0
I[9]              6.1  1.4e-3    0.1

In [5]:
with open('fit_nuts_between_strain.pkl', 'wb') as g:

    pickle.dump(fit_nuts_between_strain, g)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  This is separate from the ipykernel package so we can avoid doing imports until
